# `scippr` inference module

This notebook outlines the `scippr` inference procedure based on hierarchical inference.  To improve performance, all probabilities will be calculated as log probabilities.

In [ ]:
import numpy as np
import astropy.cosmology as cosmology
import scipy.optimize as spo
import scipy.stats as sps
from scipy.stats import norm
import scipy.linalg as la
import emcee
from datetime import datetime
import hickle
import bisect
import daft
import cProfile
import StringIO
import sys
epsilon = np.log(sys.float_info.epsilon)

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
rc("font", family="serif", size=12)
rc("text", usetex=True)
colors = 'rbgcymk'

`scippr` is based on a probabilistic graphical model, illustrated below.  The model has two types of observables, shown in shaded circles, supernova lightcurves $\underline{\ell}_{n}$ and host galaxy photometry $\vec{m}_{n}$.  The parameters, which are by definition not directly observable, are shown in empty circles.  The latent variables of supernova type $t_{n}$, redshift $z_{n}$, and distance modulus $\mu_{n}$ are parameters over which we will marginalize, without ever directly inferring them, and while all three of them influence $\underline{\ell}_{n}$, only $z_{n}$ affects $\vec{m}_{n}$ in this model.  In other words, _we currently assume no relationship between supernova type and host galaxy photometry, an assumption we may revisit in the future_.  The selection functions parametrized by $\vec{P}$, $\vec{V}$, $\vec{C}$, and $\vec{M}$ are known constants of the survey symbolized by dots that influence the possible lightcurves and host galaxy photometry that are included in the sample.  The box indicates that the latent variables and the observables are generated independently $N$ times for each supernova in the sample.  The hyperparameters we would like to estimate are the redshift-dependent supernova type proportions $\underline{\phi}$ that determine $t_{n}$ and $z_{n}$ and the cosmological parameters $\vec{\theta}$ that relate $z_{n}$ to $\mu_{n}$, which are shared by all $N$ supernovae in the observed sample.  

In [ ]:
#initialize the PGM
pgm = daft.PGM([6, 4.5], origin=[0, 0])

#desired hyperparameters
pgm.add_node(daft.Node("cosmology", r"$\vec{\theta}$", 2., 4.))
pgm.add_node(daft.Node("dist", r"$\underline{\phi}$", 3.5, 4.))
#pgm.add_node(daft.Node("rates", r"$\vec{R}$", 3., 5.5, fixed=True))

#latent variables/parameters
pgm.add_node(daft.Node("distance", r"$\mu_{n}$", 2., 2.5))
pgm.add_node(daft.Node("redshift", r"$z_{n}$", 3., 3.))
pgm.add_node(daft.Node("type", r"$t_{n}$", 4., 2.5))

#data
pgm.add_node(daft.Node("lightcurve", r"$\underline{\ell}_{n}$", 2.5, 1., observed=True))
pgm.add_node(daft.Node("photometry", r"$\vec{m}_{n}$", 3.5, 1., observed=True))

#known constant parameters
pgm.add_node(daft.Node("lightcurve selection", r"$\vec{P}, \vec{V}, \vec{C}$", 1., 1.75, fixed=True))
pgm.add_node(daft.Node("photometry selection", r"$\vec{M}$", 5., 1.75, fixed=True))

# Add in the edges.
pgm.add_edge("dist", "type")
pgm.add_edge("cosmology", "distance")
pgm.add_edge("dist", "redshift")
pgm.add_edge("redshift", "distance")
#pgm.add_edge("distance", "photometry")
pgm.add_edge("distance", "lightcurve")
pgm.add_edge("redshift", "photometry")
pgm.add_edge("redshift", "lightcurve")
pgm.add_edge("type", "lightcurve")
pgm.add_edge("photometry selection", "photometry")
pgm.add_edge("lightcurve selection", "lightcurve")

# plates
pgm.add_plate(daft.Plate([1.5, 0.5, 3., 3.], label=r"$n = 1, \cdots, N$"))

# Render and save.
pgm.render()
pgm.figure.show()

The probabilistic graphical model will guide us in characterizing the posterior distribution $\ln[p(\vec{\theta}, \underline{\phi} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N}, \vec{P}, \vec{V}, \vec{C}, \vec{M})]$ of the hyperparameters given the observed data, but as a sneak preview, this is the form it will take:

\begin{align}
\ln[p(\vec{\theta}, \underline{\phi} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N}, \vec{P}, \vec{V}, \vec{C}, \vec{M})] \propto & \ln[p(\vec{\theta}, \underline{\phi})]\\
& +\sum_{n}^{N}\ \ln[\iiint\ \exp[\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\ell}_{n}, \vec{m}_{n}, \vec{\theta}^{*}, \underline{\phi}^{*}, \vec{P}, \vec{V}, \vec{C}, \vec{M})]+\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}, \underline{\phi})]\\
\ \ \ \ \ & -\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}^{*}, \underline{\phi}^{*})]]\ d\mu_{n}\ dz_{n}\ dt_{n}]
\end{align}

## Setting up the parameter space

First we set up the parameter space for the latent variables of supernova type $t$, redshift $z$, and distance modulus $\mu$.

In [ ]:
with open('../data.hkl', 'r+') as in_file:
    sim_info = hickle.load(in_file)
    
types = sim_info['types']
n_types = len(types)

z_bins = sim_info['z_bins']
z_difs = z_bins[1:] - z_bins[:-1]
z_mids = (z_bins[1:] + z_bins[:-1]) / 2.
n_zs = len(z_difs)

mu_bins = sim_info['mu_bins']
mu_difs = mu_bins[1:] - mu_bins[:-1]
mu_mids = (mu_bins[1:] + mu_bins[:-1]) / 2.
n_mus = len(mu_difs)

def safe_log(arr, threshold=sys.float_info.epsilon):
    shape = np.shape(arr)
    flat = arr.flatten()
    logged = np.log(np.array([max(a, threshold) for a in flat])).reshape(shape)
    return logged

## Introducing the log-interim posteriors and interim hyperparameters

`scippr` requires inputs in the form of catalogs $\{\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\ell}_{n}, \vec{m}_{n}, \underline{\phi}^{*}, \vec{\theta}^{*}, \vec{P}, \vec{V}, \vec{C}, \vec{M})]\}_{N}$ of interim log-posteriors expressed as `3D` arrays constituting probabilities over $t_{n}$, $z_{n}$, and $\mu_{n}$, enabling rapid computation of the log-posterior $\ln[p(\underline{\phi}, \vec{\theta} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N}, \vec{P}, \vec{V}, \vec{C}, \vec{M})$ over the hyperparameters $\underline{\phi}$ and $\vec{\theta}$ of scientific interest. 

In [ ]:
lninterimposteriors = sim_info['interim ln posteriors']
(n_SNe, n_types, n_zs, n_mus) = np.shape(lninterimposteriors)

# these are going to get a lot narrower
fig = plt.figure(figsize=(n_types * len(colors), n_SNe * len(colors)))
p = 0
for s in range(n_SNe)[:len(colors)]:
    for t in range(n_types):
        p += 1
        plt.subplot(n_SNe, n_types, p)
        plt.pcolormesh(z_mids, mu_mids, lninterimposteriors[s][t].T, cmap='viridis')#, vmin = 0., vmax = 3.)
        plt.colorbar()
        plt.xlabel(r'$z$')
        plt.ylabel(r'$\mu$')
        plt.axis([z_bins[0], z_bins[-1], mu_bins[0], mu_bins[-1]])

The interim posteriors must always come with interim hyperparameters specifying $p^{*}(t, z, \mu)$ used to produce them.

In [ ]:
interim_ln_prior = sim_info['interim ln prior']

# interim_n_of_z_hyperparams = interimhyperparameters['phi']

# interim_cosmo_hyperparams = interimhyperparameters['theta'][0]#np.array([[interim_H0, interim_Om0], [delta_H0, delta_Om0]])
# n_cosmo_hyperparams = len(interim_cosmo_hyperparams)
# interim_cosmo_hyperparam_vars = interimhyperparameters['theta'][1]
# interim_cosmo_hyperparam_cov = interim_cosmo_hyperparam_vars * np.eye(n_cosmo_hyperparams)
# interim_dist = sps.multivariate_normal(mean = interim_cosmo_hyperparams, cov = interim_cosmo_hyperparam_cov)
# interim_cosmo = cosmology.FlatLambdaCDM(H0=interim_cosmo_hyperparams[0], Om0=interim_cosmo_hyperparams[1])

# # WMAP starting points for optimization
# guess_H0 = 70.0
# guess_Om0 = 1. - 0.721
# ivals_cosmo_hyperparams = np.array([guess_H0, guess_Om0])

# def inverter(z, mu):
#     def cosmo_helper(hyperparams):
#         return np.array([abs(cosmology.FlatLambdaCDM(H0=hyperparams[0], Om0=hyperparams[1]).distmod(z).value - mu)])
#     solved_cosmo = spo.minimize(cosmo_helper, ivals_cosmo_hyperparams, method="Nelder-Mead", options={"maxfev": 1e5, "maxiter":1e5})
#     ln_prob = interim_dist.logpdf(solved_cosmo.x)
#     return ln_prob#max(prob, sys.float_info.epsilon)

# # note the approximation of cdf[z_min, z_max] = pdf[z_mid]
# interim_sheet = np.zeros((n_zs, n_mus))
# for z in range(n_zs):
#     for mu in range(n_mus):
#         ln_prob = inverter(z_mids[z], mu_mids[mu])
#         interim_sheet[z][mu] = ln_prob
# interim_ln_prior = interim_n_of_z_hyperparams[:, np.newaxis] + interim_sheet[np.newaxis, :]
# interim_prior = np.exp(interim_ln_prior)
# interim_prior /= np.sum(interim_prior * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, np.newaxis, :])
# interim_ln_prior = safe_log(interim_prior)
# assert np.isclose(np.sum(interim_prior * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, np.newaxis, :]), 1.)

# the interim prior and truth are way too close to each other. . . but that's realistic
fig = plt.figure(figsize=(n_types*len(colors), len(colors)))
for t in range(n_types):
    plt.subplot(1, n_types, t+1)
    plt.pcolormesh(z_mids, mu_mids, interim_ln_prior[t].T, cmap='viridis')#, vmin = 0., vmax = 3.)
    plt.title('SN '+types[t]+' interim prior distribution')
    plt.xlabel(r'$z$')
    plt.ylabel(r'$\mu$')
    plt.legend(loc='lower right', fontsize='small')
    plt.axis([z_bins[0], z_bins[-1], mu_bins[0], mu_bins[-1]])
    plt.colorbar()

## Choosing the log-hyperprior probability distribution

As in any Bayesian inference, we must choose a hyperprior distribution over the hyperparameters $\vec{\theta}$ and $\underline{\phi}$ that we wish to estimate.  At this stage we will only attempt to infer $\vec{\theta}=(H_{0}, \Omega_{m0})$.  We will choose a pair of truncated Gaussians as the hyperprior distribution over the hyperparameters (i.e. assuming $H_{0}$ and $\Omega_{m, 0}$ are independent).

In [ ]:
class indie_dist(object):
    def __init__(self, in_dists):
        self.dists = in_dists
        self.n_dists = len(self.dists)
    def rvs(self, n_samps = 1):
        samps = []
        for d in range(self.n_dists):
            samps.append(self.dists[d].rvs(n_samps))
        return(np.array(samps).T)
    def logpdf(self, locs):
        n_items = len(locs)
        lnprobs = np.ones(n_items)
        locs = locs.T
        for d in range(self.n_dists):
            lnprobs += self.dists[d].logpdf(locs[d])
        return(lnprobs)

In [ ]:
# WMAP, with 10 * errors so we can see what's going on in crappy plots
wmap_H0 = 70.0
delta_H0 = 2.2 * 10.
wmap_Om0 = 1. - 0.721
delta_Om0 = 0.025 * 10.

[H0_mean, H0_std] = [wmap_H0, delta_H0]
min_H0, max_H0 = 50, 100
H0_low, H0_high = (min_H0 - H0_mean) / H0_std, (max_H0 - H0_mean) / H0_std
H0_dist = sps.truncnorm(H0_low, H0_high, loc = H0_mean, scale = H0_std)

[Om0_mean, Om0_std] = [wmap_Om0, delta_Om0]
min_Om0, max_Om0 = 0., 1.
Om0_low, Om0_high = (min_Om0 - Om0_mean) / Om0_std, (max_Om0 - Om0_mean) / Om0_std
Om0_dist = sps.truncnorm(Om0_low, Om0_high, loc = Om0_mean, scale = Om0_std)

prior_cosmo_hyperparams = np.array([H0_dist.rvs(), Om0_dist.rvs()])
cosmo_param_names = [r'$H_{0}$', r'$\Omega_{m0}$']
n_cosmo_hyperparams = len(prior_cosmo_hyperparams)
# prior_cosmo_hyperparam_vars = interim_cosmo_hyperparam_vars
# prior_cosmo_dist = sps.multivariate_normal(mean = prior_cosmo_hyperparams, cov = prior_cosmo_hyperparam_vars)
prior_cosmo_dist = indie_dist([H0_dist, Om0_dist])
prior_cosmo = cosmology.FlatLambdaCDM(H0=prior_cosmo_hyperparams[0], Om0=prior_cosmo_hyperparams[1])

# # prior on phi is gaussian about flat in log space
# prior_mean = np.ones((n_types, n_zs)) / n_types
# prior_mean /= np.sum(prior_mean * z_difs[np.newaxis, :])
# assert np.isclose(np.sum(prior_mean * z_difs[np.newaxis, :]), 1.)
# prior_mean = safe_log(prior_mean.flatten())
# prior_sigmas = np.array([0.5, 1., 0.25])
# prior_covariance = la.block_diag(prior_sigmas[0] * np.eye(n_zs), prior_sigmas[1] * np.eye(n_zs), prior_sigmas[2] * np.eye(n_zs))
# # prior_n_of_z_hyperparams = [prior_mean, prior_sigmas]
# prior_n_of_z_dist = sps.multivariate_normal(mean = prior_mean, cov = prior_covariance)

When we evaluate the log-hyperprior probability $\ln[p(\vec{\theta}, \underline{\phi})]$, we will then be evaluating the log-probability of the hyperprior distribution at the given values of the hyperparameters $\vec{\theta}$ and $\underline{\phi}$.  Thus, the log-hyperprior probability is a scalar as expected.

In [ ]:
def lnprior(hyperparameters):
    cosmo_hyperparameters = hyperparameters#['theta']
#     dist_hyperparameters = hyperparameters['phi']
    cosmo_prior_prob = prior_cosmo_dist.logpdf(cosmo_hyperparameters)
#     dist_prior_prob = prior_n_of_z.logpdf(dist_hyperparameters.flatten())
    return cosmo_prior_prob # + dist_prior_prob

## Calculating the log-hyperlikelihood

The log-hyperlikelihood $\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}, \underline{\phi})] = \ln[p(t_{n}, z_{n} | \underline{\phi})]+\ln[p(\mu_{n} | z_{n}, \vec{\theta})]$ is the sum of two terms separable in the hyperparameters.  In our parametrization, the first is derived from a constant lookup table that can be neglected for now and the second is a $\delta$ function located at the `cosmo.distmod()` function evaluated at the given redshift where `cosmo` is defined using the cosmological parameters in $\vec{\theta}$.  We will at this stage need to introduce a function mapping redshifts into distance moduli under a given cosmology.

In [ ]:
def lnhyperlikelihood(hyperparameters):
    cosmo_hyperparameters = hyperparameters#['theta']
#     dist_hyperparameters = hyperparameters['phi']
    if cosmo_hyperparameters[1] >= 0. and cosmo_hyperparameters[1] <= 1.:
        sample_cosmo = cosmology.FlatLambdaCDM(H0=cosmo_hyperparameters[0], Om0=cosmo_hyperparameters[1])
        log_delta = mu_binner(sample_cosmo.distmod(z_mids).value)
        return log_delta[np.newaxis, np.newaxis, :] # + dist_hyperparameters[:, :, np.newaxis]
    else:
        return epsilon

def mu_binner(mus):
    matrix = []
    for mu in mus:
        vector = np.zeros(n_mus)
        ind = bisect.bisect(mu_bins[:-1], mu) - 1
        vector[ind] += 1. / mu_difs[ind]
        matrix.append(vector)
    return safe_log(np.array(matrix))

Before we construct the log-posterior probability function, we note that we will be using the log-hyperlikelihood evaluated at the interim hyperparameter values at every evaluation of the log-posterior probability, so we define it as a constant now.

In [ ]:
# for_interimhyperlikelihood = {}
# for_interimhyperlikelihood['theta'] = interim_cosmo_hyperparams
# for_interimhyperlikelihood['phi'] = interimhyperparameters['phi']
# lninterimhyperlikelihood = lnhyperlikelihood(for_interimhyperlikelihood)
lninterimhyperlikelihood = interim_ln_prior#lnhyperlikelihood(interim_cosmo_hyperparams)

## Constructing the log-posterior probability

The full log-posterior probability takes the following form:
\begin{equation}
\ln[p(\vec{\theta}, \underline{\phi} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N})] \propto \ln[p(\vec{\theta}, \underline{\phi})]+\sum_{n}^{N}\ \ln[\iiint\ \exp[\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\ell}_{n}, \vec{m}_{n}, \vec{\theta}^{*}, \underline{\phi}^{*})]+\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}, \underline{\phi})]-\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}^{*}, \underline{\phi}^{*})]]\ d\mu_{n}\ dz_{n}\ dt_{n}]
\end{equation}
In words, that's the sum of the log-prior probability and the sum of the logs of the integrals over the sum of the log-interim posteriors, the log-hyperlikelihood, and the negative log-interim hyperlikelihood.

In [ ]:
def lnposterior(hyperparameters):
    in_exp = lninterimposteriors + lnhyperlikelihood(hyperparameters) - lninterimhyperlikelihood
    in_log = np.sum(np.sum(np.sum(np.exp(in_exp), axis=3), axis=2), axis=1)
    in_sum = np.log(in_log)
    return np.sum(in_sum, axis=0)

def lnhyperposterior(hyperparameters):
    return lnprior(hyperparameters) + lnposterior(hyperparameters)

# MCMC Sampler

In [ ]:
mcrandstep = 10.**-1.
nthreads = 1
nwalkers = 100
nsteps = 1000
ninit = 100
output_chain = True
emcee_chain_output = "emcee_chain_testing.dat"

In [ ]:
# will need to do gibbs sampling to get phi
def run_MCMC(lnprob):
#     init_positions = []# [np.array(theta) + mcrandstep*np.random.randn(len(theta)) for i in range(int(nwalkers))]
#     for w in range(nwalkers):
#         hyperparam_dict = {}
#         hyperparam_dict['theta'] = prior_cosmo_dist.rvs()
# #         hyperparam_dict['phi'] = prior_n_of_z_dist.rvs()
#         init_positions.append(hyperparam_dict)
#     init_vals = prior_cosmo_dist.rvs()
#     print(init_vals)
#     init_H0_dist = sps.truncnorm(H0_low, H0_high, loc = init_vals[0][0], scale = H0_std)
#     init_Om0_dist = sps.truncnorm(Om0_low, Om0_high, loc = init_vals[0][1], scale = Om0_std)
#     init_dist = indie_dist([init_H0_dist, init_Om0_dist])
    init_positions = prior_cosmo_dist.rvs(nwalkers)#init_dist.rvs(nwalkers)
    sampler = emcee.EnsembleSampler(nwalkers, n_cosmo_hyperparams, lnprob)
    burning = sampler.run_mcmc(init_positions, ninit)
    burn_in = sampler.chain
    new_init_positions = np.array([item[-1] for item in burn_in])
    sampler.reset()
    output = sampler.run_mcmc(new_init_positions, nsteps)
    chain = sampler.chain
    probs = sampler.lnprobability
    fracs = sampler.acceptance_fraction
#     acors = sampler.acor(chains)
    mcmc_outputs = {}
    mcmc_outputs['chain'] = chain
    mcmc_outputs['probs'] = probs
    mcmc_outputs['fracs'] = fracs
#     mcmc_outputs['acors'] = acors
    return mcmc_outputs
#     for i, result in enumerate(sampler.run_mcmc(init_positions, ninit, storechain=False)):
#         position = result[0]
#         lnprob_pos = result[1]
#         if (i+1) % 20 == 0:
#             print("Burn-in: {0:.1f}%, ".format(100 * float(i+1) / ninit))
#             print datetime.now()
#         if output_chain:
#             if i%10 == 0:
#                 f = open(emcee_chain_output, "a")
#                 for k in range(position.shape[0]):
#                     for j in range(position.shape[1]):
#                         f.write("{:+010.5f} ".format(position[k,j]))
#                     f.write("{:+010.5f} ".format(lnprob_pos[k]))
#                     f.write("\n")
#                 f.close()
#     return output

In [ ]:
pr = cProfile.Profile()
pr.enable()

results = run_MCMC(lnhyperposterior)

pr.disable()
s = StringIO.StringIO()
sortby = 'cumtime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

In [ ]:
for w in range(nwalkers):
    to_plot = results['probs'][w]
    plt.plot(-1.*to_plot, alpha=0.05)

In [ ]:
plt.hist(-1./results['probs'].flatten())

In [ ]:
for w in range(nwalkers):
    to_plot = results['chain'][w].T
    plt.scatter(to_plot[0], to_plot[1], alpha=0.01)#1./np.mean(results['probs'][w]))
plt.scatter(67.9, 1. - 0.693, color='r')
plt.xlabel(cosmo_param_names[0])
plt.ylabel(cosmo_param_names[1])
plt.savefig('bananas.png')

# Comparison

In [ ]:
with open('../truth.hkl', 'r+') as true_file:
    true_info = hickle.load(true_file)
    
true_phi = true_info['phi']
true_theta = true_info['theta']

In [ ]:
fig = plt.figure(figsize=(n_cosmo_hyperparams * len(colors), len(colors)))
priorvals = [H0_mean, Om0_mean]
for p in range(n_cosmo_hyperparams):
    plt.subplot(1, n_cosmo_hyperparams, p + 1)
    for w in range(nwalkers):
        to_plot = results['chain'][w].T
        plt.plot(to_plot[p], alpha=0.1)#1./np.mean(results['probs'][w]))#, to_plot[1])
    plt.plot([true_theta[p]] * nsteps, c='k', linewidth=2)
    plt.xlabel('sample number')
    plt.ylabel(cosmo_param_names[p])
plt.savefig('evolution.png')